In [8]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
device=torch.device("cuda" if torch.cuda.is_available else "cpu")

In [4]:
device

device(type='cuda')

In [1]:
import logging
import os
import argparse
import random
import json
import numpy as np
import torch
import glob
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from utiles import read_race_examples,convert_examples_to_features,accuracy,select_field
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForMultipleChoice
from pytorch_pretrained_bert.optimization import BertAdam
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from tqdm import tqdm
from transformers import BertModel, BertTokenizer



In [2]:
class RaceSequence(object):
    def __init__(self,article,question,option1,option2,option3,option4,label = None):
        self.article = article
        self.question=question
        self.options=[option1,option2,option3,option4]
        self.label = label
        

class InputSequence(object):
    def __init__(self,sequence_list,label):
        self.choices_sequence = [
            {
                'input_sequence': input_sequence,
                'segment_id': segment_id,
                'mask_id': mask_id
            }
            for input_sequence,segment_id,mask_id in sequence_list
        ]
        self.label = label


In [3]:
def to_input(input_path):
    cloze_input_list=[]
    direct_input_list=[]
    for filename in os.listdir(input_path):
        file_path=input_path+"/"+filename
        # print(file_path)
        with open(file_path,'r',encoding='utf-8') as f:
            json_text=json.loads(f.read())
            correct_answers=json_text['answers']
            options_list=json_text['options']
            questions_list=json_text['questions']
            article=json_text['article']

            for i in range(len(questions_list)):
                correct_answer=correct_answers[i]
                label=ord(correct_answer)-ord('A')
                options=options_list[i]
                question=questions_list[i]
                if '_' in question:
                    case=RaceSequence(article,question,options[0],options[1],options[2],options[3],label)
                    cloze_input_list.append(case)
                else:
                    case=RaceSequence(article,question,options[0],options[1],options[2],options[3],label)
                    direct_input_list.append(case)
    return cloze_input_list,direct_input_list

In [4]:
def input_to_sequence_question(input_list,max_length,tokenizer,is_training):

    choice_list = []
    for case in input_list:
#         print(case.article)
        article_token = tokenizer.tokenize(case.article)
        question_token = tokenizer.tokenize(case.question)
        
        sequence_list = []
        for option in case.options:
            
            question_answer_token = question_token + tokenizer.tokenize(option)
            
            while len(article_token)+len(question_answer_token)>max_length-3:
                if  len(article_token)>len(question_answer_token):
                    article_token.pop()
                else:
                    question_answer_token.pop()
            
            sequence_token = ["[CLS]"] + article_token + ["[SEP]"] + question_answer_token + ["[SEP]"]
            segment_id = [0] * (len(article_token) + 2) + [1] * (len(question_answer_token) + 1)
            input_sequence = tokenizer.convert_tokens_to_ids(sequence_token)
            mask_id = [1] * len(input_sequence)

            # Zero-pad up to the sequence length.
            for i in range(max_length-len(input_sequence)):
                input_sequence+=[0]
                mask_id+=[0]
                segment_id+=[0]

            sequence_list.append((input_sequence,segment_id,mask_id))

        label = case.label

        choice_list.append(InputSequence(sequence_list,label))

    return choice_list






def input_to_sequence_cloze(input_list,max_length,tokenizer,is_training):

    choice_list = []
    for case in input_list:
#         print(case.article)
        article_token = tokenizer.tokenize(case.article)
#         question_token = tokenizer.tokenize(case.question)
        
        sequence_list = []
        for option in case.options:
            question_answer=case.question.replace("_",option)
            
            question_answer_token = tokenizer.tokenize(question_answer)
            
            while len(article_token)+len(question_answer_token)>max_length-3:
                if  len(article_token)>len(question_answer_token):
                    article_token.pop()
                else:
                    question_answer_token.pop()
            
            sequence_token = ["[CLS]"] + article_token + ["[SEP]"] + question_answer_token + ["[SEP]"]
            segment_id = [0] * (len(article_token) + 2) + [1] * (len(question_answer_token) + 1)
            input_sequence = tokenizer.convert_tokens_to_ids(sequence_token)
            mask_id = [1] * len(input_sequence)

            # Zero-pad up to the sequence length.
            for i in range(max_length-len(input_sequence)):
                input_sequence+=[0]
                mask_id+=[0]
                segment_id+=[0]

            sequence_list.append((input_sequence,segment_id,mask_id))

        label = case.label

        choice_list.append(InputSequence(sequence_list,label))

    return choice_list


In [5]:
train_middle_path='data/train/middle'
train_high_path='data/train/high'

max_seq_length=360
is_training=True
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


train_input_cloze,train_input_direct=to_input(train_middle_path)
train_input_high_cloze,train_input_high_direct=to_input(train_high_path)
train_input_cloze+=train_input_high_cloze
train_input_direct+=train_input_high_direct
print("train_input completed!")
input_sequences_question=input_to_sequence_question(train_input_direct, max_seq_length,tokenizer,is_training)
input_sequences_cloze=input_to_sequence_cloze(train_input_cloze, max_seq_length,tokenizer,is_training)

train_input completed!


In [6]:
train_input=train_input_cloze+train_input_direct

In [7]:
input_sequences=input_sequences_question+input_sequences_cloze

In [8]:
eval_middle_path='data/dev/middle'
max_seq_length=360
is_training=False
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

eval_input_middle_cloze,eval_input_middle_direct=to_input(eval_middle_path)

input_sequences_eval_middle_cloze=input_to_sequence_cloze(eval_input_middle_cloze, max_seq_length,tokenizer,is_training)
input_sequences_eval_middle_direct=input_to_sequence_question(eval_input_middle_direct, max_seq_length,tokenizer,is_training)
input_sequences_eval_middle=input_sequences_eval_middle_cloze+input_sequences_eval_middle_direct

In [9]:
def warmup_linear(x, warmup=0.002):
    if x < warmup:
        return x/warmup
    return 1.0 - x


def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)

In [10]:
def to_all_tensor(input_sequences,key):
    all_input_sequences=[]
    for input_sequence in input_sequences:
        lst=[]
        for case in input_sequence.choices_sequence:
            lst.append(case[key])
        all_input_sequences.append(lst)
    return torch.tensor(all_input_sequences,dtype=torch.long)

In [11]:
train_input_sequence=to_all_tensor(input_sequences,'input_sequence')
train_segment_id=to_all_tensor(input_sequences,'segment_id')
train_mask_id=to_all_tensor(input_sequences,'mask_id')
train_label=torch.tensor([case.label for case in input_sequences],dtype=torch.long)

In [12]:
eval_input_sequence_middle=to_all_tensor(input_sequences_eval_middle,'input_sequence')
eval_segment_id_middle=to_all_tensor(input_sequences_eval_middle,'segment_id')
eval_mask_id_middle=to_all_tensor(input_sequences_eval_middle,'mask_id')
eval_label_middle=torch.tensor([case.label for case in input_sequences_eval_middle],dtype=torch.long)

In [13]:
eval_input_sequence_middle.shape

torch.Size([1436, 4, 360])

In [20]:
train_input_sequence.shape

torch.Size([87866, 4, 360])

In [21]:
device=torch.device("cuda" if torch.cuda.is_available else "cpu")

model = BertForMultipleChoice.from_pretrained("MiniLM-L6-H384-distilled-from-BERT-Base",
        cache_dir=os.path.join(str(PYTORCH_PRETRAINED_BERT_CACHE), 'distributed_{}'.format(-1)),
        num_choices=4).to(device)

In [22]:
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule


learning_rate=2.5e-5
warmup_proportion=0.1
batch_size=4
gradient_accumulation_steps=5
# train_batch_size=int(batch_size/gradient_accumulation_steps)
train_batch_size=batch_size
eval_batch_size=4

num_train_steps=len(train_input)

n_gpu=torch.cuda.device_count()
epoch_num=5


device=torch.device("cuda" if torch.cuda.is_available else "cpu")

model = BertForMultipleChoice.from_pretrained("MiniLM-L6-H384-distilled-from-BERT-Base",
        cache_dir=os.path.join(str(PYTORCH_PRETRAINED_BERT_CACHE), 'distributed_{}'.format(-1)),
        num_choices=4).to(device)

param_optimizer=list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]

optimizer = BertAdam(optimizer_grouped_parameters,
                    lr=learning_rate,
                    warmup=warmup_proportion,
                    t_total=num_train_steps)

# optimizer = BertAdam(model.parameters(),lr=learning_rate,schedule='warmup_linear',warmup=warmup_proportion,t_total=num_train_steps)

train_dataset=TensorDataset(train_input_sequence,train_segment_id,train_mask_id,train_label)
train_sampler=RandomSampler(train_dataset)
train_data=DataLoader(train_dataset,sampler=train_sampler, batch_size=train_batch_size)

eval_dataset=TensorDataset(eval_input_sequence_middle,eval_segment_id_middle,eval_mask_id_middle,eval_label_middle)
eval_sampler=RandomSampler(eval_dataset)
eval_data=DataLoader(eval_dataset,sampler=eval_sampler, batch_size=eval_batch_size)

In [23]:
num_train_steps

87866

In [25]:
def val_race(eval_dataloader,device,model,global_step):
    print("begin evaluation!!!")
    eval_loss, eval_accuracy = 0, 0
    eval_steps, eval_examples = 0, 0
    for step, batch in enumerate(eval_dataloader):
        batch = tuple(t.to(device) for t in batch)
        input_ids, segment_ids, input_mask, label_ids = batch

        with torch.no_grad():
            tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
            logits = model(input_ids, segment_ids, input_mask)

        logits = logits.detach().cpu().numpy()
        label_ids = label_ids.to('cpu').numpy()
        tmp_eval_accuracy = accuracy(logits, label_ids)

        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy

        eval_examples += input_ids.size(0)
        eval_steps += 1

    eval_loss = eval_loss / eval_steps
    eval_accuracy = eval_accuracy / eval_examples
    print("eval_loss:",eval_loss)
    print("eval_accuracy:",eval_accuracy)

    result = {'dev_eval_loss': eval_loss,
              'dev_eval_accuracy': eval_accuracy,
              'global_step': global_step}

    output_eval_file = os.path.join('output', "eval_results_nowarmpup.txt")
    with open(output_eval_file, "a+") as writer:
        for key in sorted(result.keys()):
            writer.write("%s = %s\n" % (key, str(result[key])))

In [26]:
def train_race(train_dataloader,device,n_gpu,model,optimizer,global_step,t_total,train_loss,train_examples,train_steps):
    for step, data in enumerate(tqdm(train_dataloader)):
        data = tuple(t.to(device) for t in data)
        input_ids, segment_ids,input_mask, label_ids = data
#         print(segment_ids)
        loss = model(input_ids, segment_ids, input_mask, label_ids)
        
        train_loss += loss.item()
        train_examples += input_ids.size(0)
        train_steps += 1

        loss.backward()

        
         # modify learning rate with special warm up BERT uses
        if (step + 1) % gradient_accumulation_steps == 0:
#             tmp_lr= learning_rate * warmup_linear(global_step / t_total, warmup_proportion)
# we do not do linear warmup to see the effect
            tmp_lr=learning_rate
#             print(warmup_proportion)
            for param_group in optimizer.param_groups:
                param_group['lr'] = tmp_lr
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
#             print("Training loss: {}, global step: {}".format(tr_loss / nb_tr_steps, global_step))

        if (global_step+1) % 100 == 0:
            print("Training loss: {}, global step: {}".format(train_loss / train_steps, global_step))
            
            

In [27]:
n_gpu

1

In [27]:
global_step=0
model.train()

for epoch in range(epoch_num):
    train_loss=0
    training_case_num=0
    training_step_num=0
    
  
  
    print("Training Epoch: {}/{}".format(epoch+1, int(epoch_num)))
    train_race(train_data,device,n_gpu,model,optimizer,global_step,num_train_steps,train_loss,training_case_num,training_step_num)
#     if (global_step+1) % 100 == 0:
    model.eval()
    val_race(eval_data, device, model, global_step)

  

  0%|          | 0/21967 [00:00<?, ?it/s]

Training Epoch: 1/5


  0%|          | 3/21967 [00:00<2:34:07,  2.38it/s]/home/zihuiliu/.local/lib/python3.8/site-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  next_m.mul_(beta1).add_(1 - beta1, grad)
  2%|▏         | 496/21967 [00:45<32:45, 10.93it/s] 

Training loss: 1.386563254847671, global step: 99
Training loss: 1.3865545988082886, global step: 99
Training loss: 1.3865566644630203, global step: 99


  2%|▏         | 500/21967 [00:45<33:00, 10.84it/s]

...

Training loss: 0.4115273549134466, global step: 4299
Training loss: 0.4115624372343126, global step: 4299
Training loss: 0.411551205558317, global step: 4299


 98%|█████████▊| 21500/21967 [30:59<00:42, 11.01it/s]

Training loss: 0.41154156364206484, global step: 4299
Training loss: 0.41152430292398123, global step: 4299


100%|██████████| 21967/21967 [31:39<00:00, 11.56it/s]


begin evaluation!!!
eval_loss: 1.2231793509367244
eval_accuracy: 0.6455431754874652


In [28]:
test_path='data/test/middle'
is_training_test=False
test_input_cloze,test_input_direct=to_input(test_path)
test_sequences_cloze=input_to_sequence_cloze(test_input_cloze,max_seq_length,tokenizer,is_training_test)
test_sequences_direct=input_to_sequence_question(test_input_direct,max_seq_length,tokenizer,is_training_test)
test_sequences=test_sequences_cloze+test_sequences_direct
test_batch_size=1

test_input_sequence=to_all_tensor(test_sequences,'input_sequence')
test_segment_id=to_all_tensor(test_sequences,'segment_id')
test_mask_id=to_all_tensor(test_sequences,'mask_id')
test_label=torch.tensor([case.label for case in test_sequences],dtype=torch.long)

test_dataset=TensorDataset(test_input_sequence,test_segment_id,test_mask_id,test_label)
test_sampler=RandomSampler(test_dataset)
test_data=DataLoader(test_dataset,sampler=test_sampler, batch_size=batch_size)





In [29]:
test_path2='data/test/high'
is_training_test=False
test_input2_cloze,test_input2_direct=to_input(test_path2)
test_sequences2_cloze=input_to_sequence_cloze(test_input2_cloze,max_seq_length,tokenizer,is_training_test)
test_sequences2_direct=input_to_sequence_question(test_input2_direct,max_seq_length,tokenizer,is_training_test)
test_sequences2=test_sequences2_cloze+test_sequences2_direct
test_batch_size=1

test_input_sequence2=to_all_tensor(test_sequences2,'input_sequence')
test_segment_id2=to_all_tensor(test_sequences2,'segment_id')
test_mask_id2=to_all_tensor(test_sequences2,'mask_id')
test_label2=torch.tensor([case.label for case in test_sequences2],dtype=torch.long)

test_dataset2=TensorDataset(test_input_sequence2,test_segment_id2,test_mask_id2,test_label2)
test_sampler2=RandomSampler(test_dataset2)
test_data2=DataLoader(test_dataset2,sampler=test_sampler2, batch_size=batch_size)

In [30]:
def test_race(test_dataloader,device,model,global_step):
    print("begin testing!!!")
    test_loss, test_accuracy = 0, 0
    nb_test_steps, nb_test_examples = 0, 0
    for step, batch in enumerate(test_dataloader):
        batch = tuple(t.to(device) for t in batch)
        input_ids, segment_ids, input_mask, label_ids = batch

        with torch.no_grad():
            tmp_test_loss = model(input_ids, segment_ids, input_mask, label_ids)
            logits = model(input_ids, segment_ids, input_mask)

        logits = logits.detach().cpu().numpy()
        label_ids = label_ids.to('cpu').numpy()
        tmp_test_accuracy = accuracy(logits, label_ids)

        test_loss += tmp_test_loss.mean().item()
        test_accuracy += tmp_test_accuracy

        nb_test_examples += input_ids.size(0)
        nb_test_steps += 1

    test_loss = test_loss / nb_test_steps
    test_accuracy = test_accuracy / nb_test_examples
    print("test_loss:",test_loss)
    print("test_accuracy:",test_accuracy)

    result = {'dev_test_loss': test_loss,
              'dev_test_accuracy': test_accuracy,
              'global_step': global_step}

    output_test_file = os.path.join('output', "test_results.txt")
    with open(output_test_file, "a+") as writer:
        for key in sorted(result.keys()):
            writer.write("%s = %s\n" % (key, str(result[key])))

In [31]:
model.eval()
test_race(test_data, device, model, global_step)

begin testing!!!
test_loss: 1.1780953962829155
test_accuracy: 0.6622562674094707


In [32]:
model.eval()
test_race(test_data2, device, model, global_step)

begin testing!!!
test_loss: 1.5713081693594078
test_accuracy: 0.5520297312750143


In [33]:
test_cloze=test_sequences_cloze+test_sequences2_cloze
test_direct=test_sequences_direct+test_sequences2_direct

test_input_sequence_cloze=to_all_tensor(test_cloze,'input_sequence')
test_segment_id_cloze=to_all_tensor(test_cloze,'segment_id')
test_mask_id_cloze=to_all_tensor(test_cloze,'mask_id')
test_label_cloze=torch.tensor([case.label for case in test_cloze],dtype=torch.long)

test_dataset_cloze=TensorDataset(test_input_sequence_cloze,test_segment_id_cloze,test_mask_id_cloze,test_label_cloze)
test_sampler_cloze=RandomSampler(test_dataset_cloze)
test_data_cloze=DataLoader(test_dataset_cloze,sampler=test_sampler_cloze, batch_size=batch_size)

test_input_sequence_direct=to_all_tensor(test_direct,'input_sequence')
test_segment_id_direct=to_all_tensor(test_direct,'segment_id')
test_mask_id_direct=to_all_tensor(test_direct,'mask_id')
test_label_direct=torch.tensor([case.label for case in test_direct],dtype=torch.long)

test_dataset_direct=TensorDataset(test_input_sequence_direct,test_segment_id_direct,test_mask_id_direct,test_label_direct)
test_sampler_direct=RandomSampler(test_dataset_direct)
test_data_direct=DataLoader(test_dataset_direct,sampler=test_sampler_direct, batch_size=batch_size)

In [34]:
model.eval()
test_race(test_data_cloze,device,model,global_step)

begin testing!!!
test_loss: 1.447483023742803
test_accuracy: 0.5973484848484848


In [35]:
model.eval()
test_race(test_data_direct,device,model,global_step)

begin testing!!!
test_loss: 1.4676962107669835
test_accuracy: 0.5688753269398431
